In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from src import bigquery, utils

In [4]:
secrets = utils.load_json("secrets.json")
bq_client = bigquery.init_bigquery_client(secrets.get("GCP_CREDENTIALS"))

In [5]:
query_kwargs = {
    "n": 10,
    "is_women": True,
    "sort_by_date": True,
    "catalog_score": 1
}

query = bigquery.query_items(**query_kwargs)

loader = bigquery.run_query(
    client=bq_client, query=query, to_list=True
)

BadRequest: 400 Column name created_at is ambiguous at [11:10]

(job ID: aa98ff02-e6e4-4b63-b199-d48b4630d9cb)

                         -----Query Job SQL Follows-----                         

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:SELECT i.id, p.point_id, i.vinted_id, i.url, i.category_type, c.score
   3:FROM `recove-450509.vinted.item_active` i
   4:INNER JOIN `recove-450509.vinted.pinecone` AS p ON i.id = p.item_id
   5:INNER JOIN `recove-450509.vinted.catalog_importance` AS c USING (catalog_id)
   6:LEFT JOIN `recove-450509.vinted.sold` AS s USING (vinted_id)
   7:WHERE s.vinted_id IS NULL
   8:    
   9:AND c.score = 1
  10:AND women = True
  11:ORDER BY created_at DESC
  12:LIMIT 10
    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |

In [6]:
loader

NameError: name 'loader' is not defined